In [8]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import datetime as dt
from data/plotting_streamlit import data_import, scraper

SyntaxError: invalid syntax (3488480166.py, line 6)

### List of mumsnet urls

In [ ]:
url_list = ['https://www.mumsnet.com/talk/am_i_being_unreasonable/4676538-if-you-like-wordle-plusword-is-even-better-thread-4?page=',
            'https://www.mumsnet.com/talk/_chat/4714295-plusword-new-thread-1?page=',
            'https://www.mumsnet.com/talk/_chat/4765702-plusword-new-thread-2?page=']

## Scraper

## Scraper initialization and df generation

In [ ]:
whole_post_list=[]

# maxiumum number of pages in thread
max_posts = 41

for url in url_list:

    whole_post_list = scraper(url, ma, whole_post_list)
            
df = pd.DataFrame(whole_post_list, columns=['user', 'date', 'time', 'text'])

### Converts 'Today' and 'Yesterday to date values, creates and sorts by timestamp

In [ ]:
df['date'] = df['date'].str.replace('Yesterday', dt.datetime.strftime((dt.datetime.today() - dt.timedelta(days=1)), '%d/%m/%Y'))
df['date'] = df['date'].str.replace('Today', dt.datetime.strftime(dt.datetime.today(), '%d/%m/%Y'))
df['load_ts'] = df['date'] + ' ' + (df['time']+':00')
df['load_ts'] = df['load_ts'] + '.000'
df

In [ ]:
df['load_ts'] = pd.to_datetime(df['load_ts'], format='%d/%m/%Y %H:%M:%S.%f')
df = df.sort_values(by=['load_ts'])
df['load_ts'] = df['load_ts'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')
df['load_ts'] = df['load_ts'].str[:-3]

In [ ]:
df

In [ ]:
df.dtypes

### Extracts times from text and adds 00: to allow it to handle hours

In [ ]:
df['text'] =df['text'].str.extract(r'(\d*\d:\d\d)')
df = df.dropna(subset='text')
df = df.copy()
df['text'] =df['text'].str.replace(r'(^\d:\d\d)', r'0\1', regex=True)
df['text'] = '00:' + df['text']

### Drops duplicate entries for users on same date, drops columns and renames

In [ ]:
df = df.copy()
df= df.drop_duplicates(subset=['user', 'date'])
df = df.drop(columns=['date', 'time'])
df = df.rename(columns={'text' : 'time'})
df = df[['load_ts', 'time', 'user']]

In [ ]:
df

### Prints timestamped csv

In [ ]:
df.to_csv('../data/historical_mumsnet_data_' + str(dt.datetime.now()) + '.csv', index=False)